# Setup Ambiente di Lavoro

Prima di tutto, è **necessario** configurare l'ambiente di lavoro. La cella seguente clonerà il repository GitHub contenente tutti i moduli Python (`.py`) e si posizionerà nella directory corretta. 

**Esegui questa cella come primo passo.**

In [ ]:
# Clona il repository e posizionati nella directory corretta
# NOTA: Assicurati che l'URL del repository e il nome del branch siano corretti.
!git clone https://github.com/devedale/snn-ids.git snn-ids
%cd snn-ids
!git checkout feat/advanced-ml-pipeline

# Pipeline ML Avanzata per Cybersecurity: Guida Completa

Questo notebook è una guida interattiva a una pipeline di machine learning **avanzata** per l'analisi di traffico di rete. Oltre a una struttura modulare, include funzionalità specifiche per dati sequenziali e strategie di validazione robuste.

**Funzionalità Chiave:**
- **Finestre Temporali**: I dati non vengono trattati come eventi isolati, ma come sequenze (finestre temporali), permettendo a modelli come gli LSTM di catturare pattern nel tempo.
- **Validazione K-Fold**: Invece di una singola divisione train/test, usiamo la validazione incrociata (K-Fold) per una stima più robusta e affidabile della performance del modello.
- **Architettura Configurabile**: Puoi scegliere tra diversi tipi di modelli (es. `dense` vs `lstm`) direttamente dal file di configurazione.
- **Massima Configurabilità**: Ogni aspetto, dalla dimensione della finestra alla strategia di validazione, è controllato dal file `config.py`.

## 1. Setup Iniziale

Installiamo le librerie necessarie.

In [ ]:
# Installa le dipendenze
!pip install pandas scikit-learn tensorflow faker

## 2. Analisi del File di Configurazione (`config.py`)

Prima di iniziare, diamo un'occhiata alle nuove potenti opzioni che abbiamo introdotto in `config.py`.

### `PREPROCESSING_CONFIG`
- `use_time_windows` (bool): Se `True`, la pipeline trasforma i dati in sequenze. Essenziale per modelli come LSTM.
- `window_size` (int): Il numero di eventi (righe) in ogni sequenza/finestra. Una finestra più grande cattura pattern a lungo termine, ma richiede più memoria.
- `step` (int): Di quanti eventi si sposta la finestra. Se `step < window_size`, le finestre si sovrappongono, generando più dati di training e aiutando il modello a imparare transizioni fluide.

### `TRAINING_CONFIG`
- `validation_strategy` (str): Scegli come validare il modello.
  - `'train_test_split'`: Veloce, ma meno affidabile. Divide i dati una sola volta.
  - `'k_fold'`: Più robusto. Divide i dati `k` volte, addestrando e testando il modello su ogni divisione per ottenere una media delle performance. Ideale per avere una stima realistica delle prestazioni.
- `k_fold_splits` (int): Il numero di `k` (folds) da usare.
- `model_type` (str): Scegli l'architettura.
  - `'lstm'`: Una Rete Neurale Ricorrente, perfetta per dati sequenziali (le nostre finestre temporali).
  - `'dense'`: Una rete neurale standard. Richiede che `use_time_windows` sia `False` o che i dati vengano appiattiti.

## 3. Esecuzione della Pipeline Avanzata

### Fase 1: Generazione Dati con Timestamp
Eseguiamo lo script per creare dati sintetici, ora con una colonna `timestamp`.

In [ ]:
from create_synthetic_data import generate_synthetic_data
generate_synthetic_data()

### Fase 2: Preprocessing in Finestre Temporali
Eseguiamo il preprocessing. Notare la shape 3D dell'output `X`, che è `(numero_di_finestre, dimensione_finestra, numero_feature)`.

In [ ]:
from preprocessing.process import preprocess_data

X, y = preprocess_data()

if X is not None:
    print("\n--- Dati Processati ---")
    print(f"Shape di X: {X.shape}")
    print(f"Shape di y: {y.shape}")

### Fase 3: Training con K-Fold e LSTM
Eseguiamo il training usando le strategie definite in `config.py` (di default, K-Fold e LSTM).

In [ ]:
from training.train import train_and_evaluate

_, _ = train_and_evaluate()

### Fase 4: Predizione su una Finestra di Dati
Per fare una predizione, ora dobbiamo fornire un'intera finestra di dati.

In [ ]:
from prediction.predict import predict_on_window
from config import PREPROCESSING_CONFIG

# Creiamo una finestra di dati fittizia
window_size = PREPROCESSING_CONFIG.get('window_size', 10)
sample_window = [
    {
        "ip_sorgente": "192.168.1.10", "ip_destinazione": "10.0.0.5",
        "porta_sorgente": 12345, "porta_destinazione": 443,
        "protocollo": "UDP", "byte_inviati": 250, "byte_ricevuti": 1800
    } for _ in range(window_size)
]

prediction_label = predict_on_window(sample_window)

if prediction_label:
    print(f"\nRISULTATO FINALE: La finestra di dati è stata classificata come: '{prediction_label}'")

## 4. Guida all'Estensione e Personalizzazione

Questa sezione è identica a quella della versione precedente, dimostrando che la flessibilità del `config.py` è stata mantenuta anche con l'aggiunta delle nuove funzionalità complesse.